<a href="https://colab.research.google.com/github/norisuke3/langchain/blob/main/Build_an_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-community langgraph langchain-openai tavily-python


[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import getpass
import os

if os.environ["LANGCHAIN_API_KEY"] == "":
    print("enter LANGCHAIN_API_KEY:")
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
    os.environ["LANGCHAIN_TRACING_V2"] = "true"

if os.environ["OPENAI_API_KEY"] == "":
    print("enter OPENAI_API_KEY:")
    os.environ["OPENAI_API_KEY"] = getpass.getpass()


In [3]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMMathChain
from langchain.agents import Tool, tool
import requests
import json

model = ChatOpenAI(model="gpt-4o")

llm_math = LLMMathChain.from_llm(llm=model)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)

@tool
def naming():
    """Useful for naming a new baby"""
    
    return "Foo"

@tool
def notes() -> str:
    """Useful for reading contents of a Nori\'s note."""
    
    return requests.get('http://localhost:3000/posts').content.decode('utf-8')

@tool
def write_note(content: str) -> str:
    """Useful for writing a content to Nori's note."""
    
    data = { "title": "From AI", "body": content }
    return requests.post('http://localhost:3000/posts', json=data).content.decode('utf-8')
    
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool, naming, notes, write_note]

In [7]:
# Import relevant functionality

from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    #{"messages": [HumanMessage(content="create a new name for my baby.")]}, config
    #{"messages": [HumanMessage(content="read contents of Nori's note.")]}, config
    #{"messages": [HumanMessage(content="How do you know which tool to call?")]}, config
    #{"messages": [HumanMessage(content="What is the 'nots' tool?")]}, config
    #{"messages": [HumanMessage(content="Tell me each tool names.")]}, config
    #{"messages": [HumanMessage(content="のりさんのノートに、'こんにちは。僕はエージェントだよ。'と記して。")]}, config
    #{"messages": [HumanMessage(content="你可以在Nori的笔记上写下 “嘿，我是代理人！！我能理解任何语言！！” 吗？")]}, config
    {"messages": [HumanMessage(content="'안녕, 나는 에이전트야!! 나는 모든 언어를 이해할 수 있어!!'를 노리의 노트에 써 줄 수 있어?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uGhjusCnPSRh41le0kzPXxQp', 'function': {'arguments': '{"content":"안녕, 나는 에이전트야!! 나는 모든 언어를 이해할 수 있어!!"}', 'name': 'write_note'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 146, 'total_tokens': 179}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_ce0793330f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a0bede0b-07ac-48b8-bec1-a20272220046-0', tool_calls=[{'name': 'write_note', 'args': {'content': '안녕, 나는 에이전트야!! 나는 모든 언어를 이해할 수 있어!!'}, 'id': 'call_uGhjusCnPSRh41le0kzPXxQp'}], usage_metadata={'input_tokens': 146, 'output_tokens': 33, 'total_tokens': 179})]}}
----


ConnectionError: HTTPConnectionPool(host='localhost', port=3000): Max retries exceeded with url: /posts (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x117126310>: Failed to establish a new connection: [Errno 61] Connection refused'))